In [1]:
import re
import numpy as np
import pandas as pd
import math
import pickle
from collections import Counter

In [2]:
stopwords=["अंदर","अत","अदि","अप","अपना","अपनि","अपनी","अपने","अभि","अभी","आदि","आप","इंहिं","इंहें","इंहों","इतयादि","इत्यादि","इन","इनका","इन्हीं","इन्हें","इन्हों","इस","इसका","इसकि","इसकी","इसके","इसमें","इसि","इसी","इसे","उंहिं","उंहें","उंहों","उन","उनका","उनकि","उनकी","उनके","उनको","उन्हीं","उन्हें","उन्हों","उस","उसके","उसि","उसी","उसे","एक","एवं","एस","एसे","ऐसे","ओर","और","कइ","कई","कर","करता","करते","करना","करने","करें","कहते","कहा","का","काफि","काफ़ी","कि","किंहें","किंहों","कितना","किन्हें","किन्हों","किया","किर","किस","किसि","किसी","किसे","की","कुछ","कुल","के","को","कोइ","कोई","कोन","कोनसा","कौन","कौनसा","गया","घर","जब","जहाँ","जहां","जा","जिंहें","जिंहों","जितना","जिधर","जिन","जिन्हें","जिन्हों","जिस","जिसे","जीधर","जेसा","जेसे","जैसा","जैसे","जो","तक","तब","तरह","तिंहें","तिंहों","तिन","तिन्हें","तिन्हों","तिस","तिसे","तो","था","थि","थी","थे","दबारा","दवारा","दिया","दुसरा","दुसरे","दूसरे","दो","द्वारा","न","नहिं","नहीं","ना","निचे","निहायत","नीचे","ने","पर","पहले","पुरा","पूरा","पे","फिर","बनि","बनी","बहि","बही","बहुत","बाद","बाला","बिलकुल","भि","भितर","भी","भीतर","मगर","मानो","मे","में","यदि","यह","यहाँ","यहां","यहि","यही","या","यिह","ये","रखें","रवासा","रहा","रहे","ऱ्वासा","लिए","लिये","लेकिन","व","वगेरह","वरग","वर्ग","वह","वहाँ","वहां","वहिं","वहीं","वाले","वुह","वे","वग़ैरह","संग","सकता","सकते","सबसे","सभि","सभी","साथ","साबुत","साभ","सारा","से","सो","हि","ही","हुअ","हुआ","हुइ","हुई","हुए","हे","हें","है","हैं","हो","होता","होति","होती","होते","होना","होने"]

In [3]:
alphabets=['ऄ','अ','आ','इ','ई','उ','ऊ','ऋ','ऌ','ऍ','ऎ','ए','ऐ','ऑ','ऒ','ओ','औ','क','ख','ग','घ','ङ','च','छ','ज','झ','ञ','ट',
'ठ','ड','ढ','ण','त','थ','द','ध','न','ऩ','प','फ','ब','भ','म','य','र','ऱ','ल','ळ','ऴ','व','श','ष','स','ह']

matra=['ऀ','ँ','ं','ः','ऺ','ऻ','़','ा','ि','ी','ु','ू','ृ','ॄ','ॅ','ॆ','े','ै','ॉ','ॊ','ो','ौ','्','ॎ','ॏ','ॕ','ॖ','ॗ']

In [4]:
temp=open('pickles/posting_list.pkl',"rb")
posting_lists=pickle.load(temp)

temp=open('pickles/file_idx.pkl','rb')
file_index=pickle.load(temp)

In [5]:
unique_words=set(posting_lists.keys())

In [6]:
def remove_special_characters(text):
    string=''
    for i in text:
        if i in alphabets+matra:
            string=string+i
        else:
            string=string+" "
    return " ".join(string.split())

In [7]:
data=pd.read_csv('../hindi.csv')

In [11]:
def brs(query,counter=10):
    text = remove_special_characters(query)
    words = text.split()
    words=[word for word in words if word not in stopwords]
    
    if len(words)>1:
        query_words=[words[0]]
    else:
        query_words=words

    for i in range(1,len(words)):
        if words[i] not in ["और","या"]:
            if query_words[-1] not in ["और","या"]:
                query_words.append("और")
                query_words.append(words[i])
            else:
                query_words.append(words[i])
        elif query_words[-1] not in ["और","या"]:
            query_words.append(words[i])
    
    operators=[]
    main_words=[]
    for w in query_words:
        if w in ["और","या"]:
            operators.append(w)
        else:
            main_words.append(w)
            
    
    n=len(file_index)
    word_vector=[]
    word_vector_matrix=[]

    for w in main_words:
        word_vector=[0]*n
        if w in unique_words:
            for x in posting_lists[w].keys():
                word_vector[x]=1
        word_vector_matrix.append(word_vector)
        
        
    for w in operators:
        vector1=word_vector_matrix[0]
        vector2=word_vector_matrix[1]

        if w=="और":
            result=[b1&b2 for b1,b2 in zip(vector1,vector2)]

            word_vector_matrix.pop(0)
            word_vector_matrix.pop(0)

            word_vector_matrix.insert(0,result)
        else:
            result=[b1|b2 for b1,b2 in zip(vector1,vector2)]

            word_vector_matrix.pop(0)
            word_vector_matrix.pop(0)

            word_vector_matrix.insert(0,result)
    
    
    final_word_vector=word_vector_matrix[0]
    index=0
    slokes=[]
    description=[]
    verse_index=[]
    for i in final_word_vector:
        if not counter:
            break
        if i==1:
            verse_index.append(index)
            counter=counter-1

        index+=1
    return verse_index


In [18]:
verse_index=brs('भीष्म पितामह अपने भतीजे दुर्योधन')
data.loc[verse_index]

,Chapter,Verse,Sloke,Commentary
9,1,12,तस्य संजनयन्हर्षं कुरुवृद्धः पितामहः।\n\nसिंहन...,भीष्म पितामह अपने भतीजे दुर्योधन के भीतर के भय...
